In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-10-26 03:38:39--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?5dqZWilVJPKzS23pYccvJn8OnF123fJATuYCGoKq_j4UEAirn3x3LNiaJsLTklczrBuzXnmyyZIzczvTVCEixsLPPWz2YtkBzflTRcdOPTUvEezJWNEZ3AEH87nK0YCcXrzuYHuhgvRTS80cvK5_lg8KXpTdfbZIZRvbnihF-NPXw_fzFMhEpaLSRKOyBatz8ONNjqtKPItT7Oe_Jnw [following]
--2020-10-26 03:38:39--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?5dqZWilVJPKzS23pYccvJn8OnF123fJATuYCGoKq_j4UEAirn3x3LNiaJsLTklczrBuzXnmyyZIzczvTVCEixsLPPWz2YtkBzflTRcdOPTUvEezJWNEZ3AEH87nK

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-m59xyq4e
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-m59xyq4e
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=9b10b9626849581d370ca63002f9c21e930fd2193dac5a6a58d66879f66c5e13
  Stored in directory: /tmp/pip-ephem-wheel-cache-n79qhwcz/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <assert.h>
#include <math.h>
#include <time.h>
#include <stdint.h>
#include <sys/time.h>
#include <cuda_runtime.h>
#define TOL 0.001
#define BLOCK_DIM_VEC 32
#define NB_ELEM_MAT 32
#define BLOCK_SIZE_MAT 32

#ifndef HELPER_H_
#define HELPER_H_
#define SIZE 10
#define EPS 1e-14
#define MAX_ITER 1000

#define A(row,col) (A[(row)*SIZE + (col)])
#define b(x) (b[(x)])

float* generateA();
float* generateb();
void printVec(float* b);
void printMat(float* A);
float getMaxDiffSquared(float* a, float* b);
long getMicrotime();

#endif /* HELPER_H_ */

#ifndef SEQUENTIAL_H_
#define SEQUENTIAL_H_

void solveCG_seq(float* A, float* b, float* x);
int moreOrLessEqual(float* a, float* b);

#endif /* SEQUENTIAL_H_ */

/*
 * Prints a formated (square) matrix
 * Input: pointer to 1D-array-stored matrix (row major)
 */
void printMat(float* A) {
	int i;
	for (i = 0; i < SIZE * SIZE; i++) {
		printf("%.3e ", A[i]);
		if ((i + 1) % SIZE == 0) {
			printf("\n");
		}
	}
	printf("\n");
}

/*
 * Prints a formated vector
 * Input: pointer to 1D-array-stored vector
 */
void printVec(float* b) {
	printf("__begin_vector__\n");
	int i;
	for (i = 0; i < SIZE; i++) {
		if (b[i] > 0) {
			printf("+%.6e\n", b[i]);
		}
		else {
			printf("%.6e\n", b[i]);
		}
	}
	printf("__end_vector__\n");
}

float* generateA() {
	int i, j;
	float* A = (float*)malloc(sizeof(float) * SIZE * SIZE);

	float temp;
	for (i = 0; i < SIZE; i++) {
		for (j = 0; j <= i; j++) {
			temp = (float)rand()/RAND_MAX;
			if (i == j) {
				A(i,j) = temp + SIZE;
			}
			else {
				A(i,j) = temp;
				A(j,i) = temp;
			}
		}
	}
	return A;
}

/*
 * Returns the time in microseconds
 * Taken from https://gist.github.com/sevko/d23646ba07c77c15fde9
 */
long getMicrotime(){
	struct timeval currentTime;
	gettimeofday(&currentTime, NULL);
	return currentTime.tv_sec * (int)1e6 + currentTime.tv_usec;
}

/*
 * Generates a random vector of size SIZE
 */
float* generateb() {
	int i;
	float* b = (float*)malloc(sizeof(float) * SIZE);
	for (i = 0; i < SIZE; i++) {
		b[i] = (float)rand()/RAND_MAX;
	}
	return b;
}

float getMaxDiffSquared(float* a, float* b) {
	float max = 0.0;
	float tmp;
	int i;
	for (i = 0; i < SIZE; i++) {
		tmp = (a[i]-b[i])*(a[i]-b[i]);
		if (tmp > max) {
			max = tmp;
		}
	}
	return max;
}

/*
 * Computes a (square) matrix vector product
 * Input: pointer to 1D-array-stored matrix (row major), 1D-array-stored vector
 * Stores the product in memory at the location of the pointer out
 */
void matVecSeq(float* A, float* b, float* out) {
	int i, j;
	for (j = 0; j < SIZE; j++) {
		out[j] = 0;
		for (i = 0; i < SIZE; i++) {
			out[j] += A(j,i) * b(i);
		}
	}
}

/*
 * Computes the scalar product of 2 vectors
 * Input: pointer to 1D-array-stored vector, pointer 1D-array-stored vector
 * Output: float scalar product
 */
float vecVec(float* vec1, float* vec2) {
	int i;
	float product = 0;
	for (i = 0; i < SIZE; i++) {
		product += vec1[i] * vec2[i];
	}
	return product;
}

/*
 * Computes the sum of 2 vectors
 * Input: pointer to 1D-array-stored vector, pointer to 1D-array-stored vector
 * Stores the sum in memory at the location of the pointer out
 */
void vecPlusVecSeq(float* vec1, float* vec2, float* out) {
	int i;
	for (i = 0; i < SIZE; i++) {
		out[i] = vec1[i] + vec2[i];
	}
}

/*
 * Computes a scalar vector product
 * Input: scalar, pointer to 1D-array-stored vector
 * Stores the product in memory at the location of the pointer out
 */
void scalarVec(float alpha, float* vec2, float* out) {
	int i;
	for (i = 0; i < SIZE; i++) {
		out[i] = alpha * vec2[i];
	}
}

/*
 * Computes a scalar (square) matrix vector product
 * Input: scalar, pointer to 1D-array-stored matrix (row major), pointer to 1D-array-stored vector
 * Stores the product in memory at the location of the pointer out
 */
void scalarMatVecSeq(float alpha, float* A, float* b, float* out) {
	int i, j;
	for (j = 0; j < SIZE; j++) {
		out[j] = 0;
		for (i = 0; i < SIZE; i++) {
			out[j] += alpha * A(j,i) * b(i);
		}
	}
}

/*
 * Computes the 2-norm of a vector
 * Input: pointer to 1D-array-stored vector
 * Output: value of the norm of the vector
 */
float norm2d(float* a) {
	return sqrt(vecVec(a, a));
}


/*
 * Checks if 2 vectors or matrices are equal up to some precision
 * Input: 2 1D-array-stored vector or 2 1D-array-stored matrices
 * Output: true if same (up to some precision) else false
 */
int moreOrLessEqual(float* a, float* b) {
	int i;
	for (i = 0; i < SIZE; i++) {
		if (fabs(a[i] - b[i]) > TOL) {
			return 0;
		}
	}
	return 1;
}

/*
 * Solve the system Ax=b using the CG method
 * Input: pointer to 1D-array-stored matrix, pointer to 1D-array-stored vector, pointer to 1D-array-stored vector
 * float* x is used as initial condition and the final output is written there as well
 */
void solveCG_seq(float* A, float* b, float* x) {
	// Initialize temporary variables
	float* p = (float*) calloc(sizeof(float), SIZE);
	float* r = (float*) calloc(sizeof(float), SIZE);
	float* temp = (float*) calloc(sizeof(float), SIZE);
	float beta, alpha, rNormOld = 0.0;
	float rNorm = 1.0;
	int k = 0;
	// Set initial variables
	scalarMatVecSeq(-1.0, A, x, temp);
	vecPlusVecSeq(b, temp, r);
	scalarVec(1.0, r, p);
	rNormOld = vecVec(r, r);
	long micro_begin_seq = getMicrotime();
	while ((rNorm > EPS) && (k < MAX_ITER)) {
		matVecSeq(A, p, temp); // temp = A* p (only compute matrix vector product once)
		alpha = rNormOld / vecVec(p, temp); // alpha_k = ...
		scalarVec(-alpha, temp, temp); // r_{k+1} = ...
		vecPlusVecSeq(r, temp, r);
		scalarVec(alpha, p, temp);  // x_{k+1} = ...
		vecPlusVecSeq(x, temp, x);
		rNorm = vecVec(r, r);  // beta_k = ...
		beta = rNorm / rNormOld;
		scalarVec(beta, p, temp);  // p_{k+1} = ...
		vecPlusVecSeq(r, temp, p);
		rNormOld = rNorm;  // set rOld to r
		k++;
	}
	long micro_end_seq = getMicrotime();
	printf("\nTime spent by seq  [s]: %e\n", (float) ((micro_end_seq - micro_begin_seq)/k) / 1e6);
	// free temporary memory
	free(p);
	free(r);
	free(temp);}

__global__ void matVec(float* A, float* b, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		float tmp = 0;
		for (int i = 0; i < SIZE; i++) {
			tmp += b[i] * A[SIZE * index_x + i];
		}
		out[index_x] = tmp;
	}
}

__global__ void vecPlusVec(float* a, float* b, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		out[index_x] = b[index_x] + a[index_x];
	}
  //printf("\n%.3e",out[index_x]);
}


__global__ void vecPlusVec2(float* a, float* b, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		out[index_x] = b[index_x] + a[index_x];
		b[index_x] = 0.0;
	}
}

__global__ void matVec2(float* A, float* b, float* out) {
	__shared__ float b_shared[NB_ELEM_MAT];

	int effective_block_width;
	if ((blockIdx.x + 1) * NB_ELEM_MAT <= SIZE) {
		effective_block_width = NB_ELEM_MAT;
	}
	else {
		// needed to avoid overflow in next row
		effective_block_width = SIZE % NB_ELEM_MAT;
	}

	if (threadIdx.x < effective_block_width)
		b_shared[threadIdx.x] = b[blockIdx.x * NB_ELEM_MAT + threadIdx.x];

	__syncthreads();

	int idy = blockIdx.y * BLOCK_SIZE_MAT + threadIdx.x;
	float tmp_scal = 0.0;
	// threads outside matrix dimension are not needed (vertical)
	if (idy < SIZE) {
		for (int i = 0; i < effective_block_width; i++) {
			// take advantage of symmetric matrix for coalesced memory access
			tmp_scal += b_shared[i] * A(blockIdx.x * NB_ELEM_MAT + i, idy);
		}
		atomicAdd(out + idy, tmp_scal);
	}
}



/*
 * Computes the difference of 2 vectors
 * Input: pointer to 1D-array-stored vector, pointer to 1D-array-stored vector
 * Stores the sum in memory at the location of the pointer out
 */
__global__ void vecMinVec(float* a, float* b, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		out[index_x] = a[index_x] - b[index_x];
	}
}

/*
 * --Naive implementation--
 * Computes the inner product of 2 vectors
 * Input: pointer to 1D-array-stored vector, pointer to 1D-array-stored vector
 * Stores the product in memory at the location of the pointer out
 */
__global__ void vecVec(float* a, float* b, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	float tmp = 0.0;
	if (index_x == 0) {
		for (int i = 0; i < SIZE; i++) {
			tmp += b[i] * a[i];
		}
		*out = tmp;
	}
}

/*
 * --More efficient implementation--
 * Computes the inner product of 2 vectors
 * Input: pointer to 1D-array-stored vector, pointer to 1D-array-stored vector
 * Stores the product in memory at the location of the pointer out
 */
__global__ void vecVec2(float* a, float* b, float* out) {
	// each block has it's own shared_tmp of size BLOCK_DIM_VEC
	__shared__ float shared_tmp[BLOCK_DIM_VEC];

	// needed for atomicAdd
	if (threadIdx.x + blockDim.x * blockIdx.x == 0) {
		*out = 0.0;
	}
	if (blockIdx.x * blockDim.x + threadIdx.x < SIZE) {
		shared_tmp[threadIdx.x] = a[blockIdx.x * blockDim.x + threadIdx.x]
				* b[blockIdx.x * blockDim.x + threadIdx.x];
	} else {
		// needed for the reduction
		shared_tmp[threadIdx.x] = 0.0;
	}
	// reduction within block
	for (int i = blockDim.x / 2; i >= 1; i = i / 2) {
		// threads access memory position written by other threads so sync is needed
		__syncthreads();
		if (threadIdx.x < i) {
			shared_tmp[threadIdx.x] += shared_tmp[threadIdx.x + i];
		}
	}

	// atomic add the partial reduction in out
	if (threadIdx.x == 0) {
		atomicAdd(out, shared_tmp[0]);
	}
}

/*
 * Computes the product of a scalar with a vector
 * Input: pointer to scalar, pointer to 1D-array-stored vector
 * Stores the sum in memory at the location of the pointer out
 */
__global__ void scalarVec(float* scalar, float* a, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		out[index_x] = a[index_x] * *scalar;
	}
}

__global__ void memCopy(float* in, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x < SIZE) {
		out[index_x] = in[index_x];
	}
}

__global__ void divide(float* num, float* den, float* out) {
	unsigned int index_x = blockIdx.x * blockDim.x + threadIdx.x;
	if (index_x == 0) {
		*out = *num / *den;
	}
}



/*
 * Main CG solver
 * All the given pointers are device pointers, with correct initial values
 */
void solveCG_cuda(float* A, float* b, float* x, float* p, float* r, float* temp,
		float* alpha, float* beta, float* r_norm, float* r_norm_old,
		float* temp_scal, float* h_x, float* h_r_norm) {

	dim3 vec_block_dim(BLOCK_DIM_VEC);
	dim3 vec_grid_dim((SIZE + BLOCK_DIM_VEC - 1) / BLOCK_DIM_VEC);

	dim3 mat_grid_dim((SIZE + NB_ELEM_MAT - 1) / NB_ELEM_MAT, (SIZE + BLOCK_SIZE_MAT - 1) / BLOCK_SIZE_MAT);
	dim3 mat_block_dim(BLOCK_SIZE_MAT);

	vecVec2<<<vec_grid_dim, vec_block_dim>>>(r, r, r_norm_old);
	int k = 0;
	long micro_begin_gpu = getMicrotime();
	while ((k < MAX_ITER) && (*h_r_norm > EPS)) {
    
		// temp = A * p (only compute matrix vector product once)
		matVec2<<<mat_grid_dim, mat_block_dim>>>(A, p, temp);

		// alpha_k = ...
		vecVec2<<<vec_grid_dim, vec_block_dim>>>(p, temp, temp_scal);
		divide<<<1, 1>>>(r_norm_old, temp_scal, alpha);

		
		scalarVec<<<vec_grid_dim, vec_block_dim>>>(alpha, temp, temp);  // r_{k+1} = ...
		vecMinVec<<<vec_grid_dim, vec_block_dim>>>(r, temp, r);
		scalarVec<<<vec_grid_dim, vec_block_dim>>>(alpha, p, temp);  // x_{k+1} = ...
		vecPlusVec<<<vec_grid_dim, vec_block_dim>>>(x, temp, x);
    vecVec2<<<vec_grid_dim, vec_block_dim>>>(r, r, r_norm);  // beta_k = ...
		divide<<<1, 1>>>(r_norm, r_norm_old, beta);
		scalarVec<<<vec_grid_dim, vec_block_dim>>>(beta, p, temp);  // p_{k+1} = ...
		vecPlusVec2<<<vec_grid_dim, vec_block_dim>>>(r, temp, p);
		memCopy<<<1, 1>>>(r_norm, r_norm_old);  // set r_norm_old to r_norm
		// copy to r_norm to CPU (to evaluate stop condition)    
		cudaMemcpy(h_r_norm, r_norm, sizeof(float), cudaMemcpyDeviceToHost);
		k++;}
	long micro_end_gpu = getMicrotime();
printf("\nTime spent by gpu [s]: %e\n", (float) ((micro_end_gpu - micro_begin_gpu)/k) / 1e6);
}  

void ErrorCheck(cudaError_t status) { 
  printf("\nKernel failed!! %s", cudaGetErrorString(status));
}


////////////////////////////////////////////////////////////////////////////////
// MAIN
////////////////////////////////////////////////////////////////////////////////
int main() {
	// allocate host memory
	float* h_A = generateA();
	float* h_b = generateb();
	//printMat(h_A);
	//printVec(h_b);
	float* h_x = (float *) calloc(SIZE, sizeof(float));
  float* h_r_norm = (float *) malloc(sizeof(float));
	*h_r_norm = 1.0;

	// allocate device memory
	float* d_A;
	float* d_b;
	float* d_x;
	float* d_p;
	float* d_r;
	float* d_temp;
	cudaMalloc((void **) &d_A, SIZE * SIZE * sizeof(float));
	cudaMalloc((void **) &d_b, SIZE * sizeof(float));
	cudaMalloc((void **) &d_x, SIZE * sizeof(float));
	cudaMalloc((void **) &d_p, SIZE * sizeof(float));
	cudaMalloc((void **) &d_r, SIZE * sizeof(float));
	cudaMalloc((void **) &d_temp, SIZE * sizeof(float));

	// copy host memory to device
	cudaMemcpy(d_A, h_A, SIZE * SIZE * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, h_b, SIZE * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(d_x, h_x, SIZE * sizeof(float), cudaMemcpyHostToDevice);
	cudaError_t status = cudaGetLastError();
  if (status != cudaSuccess)
  {
      ErrorCheck(status);
      return false;
  }
	// assume x0 = 0
	cudaMemcpy(d_p, h_b, SIZE * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(d_r, h_b, SIZE * sizeof(float), cudaMemcpyHostToDevice);

	// 4 floats needed
	float* d_beta;
	float* d_alpha;
	float* d_r_norm;
	float* d_r_norm_old;
	float* d_temp_scal;
	cudaMalloc((void **) &d_beta, sizeof(float));
	cudaMalloc((void **) &d_alpha, sizeof(float));
	cudaMalloc((void **) &d_r_norm, sizeof(float));
	cudaMalloc((void **) &d_r_norm_old, sizeof(float));
	cudaMalloc((void **) &d_temp_scal, sizeof(float));

	// run the main function
	solveCG_cuda(d_A, d_b, d_x, d_p, d_r, d_temp, d_alpha, d_beta, d_r_norm,d_r_norm_old, d_temp_scal, h_x, h_r_norm);

	// allocate memory for the result on host side
	cudaDeviceSynchronize();
	// copy result from device to host
	cudaMemcpy(h_x, d_x, SIZE * sizeof(float), cudaMemcpyDeviceToHost);
  //printf("\nError:%s",e);
  printf("\nResult of parallel Conjugate Gradient:");
  for(int i=0;i<SIZE;i++){
      printf("\n%.6e",h_x[i]);
  }
	// compare output with sequential version
	float* h_x_seq = (float *) calloc(SIZE, sizeof(float));
	solveCG_seq(h_A, h_b, h_x_seq);
  printf("\nResult of serial Conjugate Gradient:");
  for(int i=0;i<SIZE;i++){
      printf("\n%.6e",h_x_seq[i]);
  }
    if(moreOrLessEqual(h_x, h_x_seq) == 1)
    printf("\nAssertion passed!");
    else printf("\nAssertion not passed");

	// cleanup memory host
	free(h_A);
	free(h_b);
	free(h_x);
	free(h_r_norm);

	// cleanup memory device
	cudaFree(d_A);
	cudaFree(d_b);
	cudaFree(d_x);
	cudaFree(d_p);
	cudaFree(d_r);
	cudaFree(d_temp);
	cudaFree(d_alpha);
	cudaFree(d_beta);
	cudaFree(d_r_norm);
	cudaFree(d_r_norm_old);
	cudaFree(d_temp_scal);

	return 0;
}



Time spent by gpu [s]: 6.400000e-05

Result of parallel Conjugate Gradient:
1.944396e-02
-6.319873e-03
-1.483612e-02
3.171908e-02
-8.702797e-03
1.399001e-02
8.913465e-02
7.072096e-02
6.900794e-02
1.930267e-02
Time spent by seq  [s]: 1.000000e-06

Result of serial Conjugate Gradient:
1.944396e-02
-6.319874e-03
-1.483612e-02
3.171908e-02
-8.702799e-03
1.399001e-02
8.913464e-02
7.072096e-02
6.900794e-02
1.930267e-02
Assertion passed!
